![](../img/330-banner.png)

# Tutorial 8

UBC 2024-26

## Outline

During this tutorial, you will 

All questions can be discussed with your classmates and the TAs - this is not a graded exercise!

### Imports

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

plt.rcParams["font.size"] = 12
from datetime import datetime

DATA_DIR = os.path.join(os.path.abspath(".."), "data/")

## Time series analysis on a more complicated dataset 

For this exercise, we will use the [rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) dataset. Our goal is to predict whether or not it will rain tomorrow based on today's measurements.

In [ ]:
rain_df = pd.read_csv(DATA_DIR + "weatherAUS.csv")
rain_df.head()

In [ ]:
rain_df.shape

**Questions of interest**

- Can we **forecast** into the future? Can we predict whether it's going to rain tomorrow?
    - The target variable is `RainTomorrow`. The target is categorical and not continuous in this case. 
- Can the date/time features help us predict the target value?


### Exploratory data analysis

We are doing some basic EDA to help you familiarize with the dataset - check our results below.

In [ ]:
rain_df.info()

In [ ]:
rain_df.describe(include="all")

- A number of missing values. 
- Some target values are also missing. Let's drop these rows. 

In [ ]:
rain_df = rain_df[rain_df["RainTomorrow"].notna()]
rain_df.shape

### Parsing datetimes 

- In general, datetimes are a huge pain! 
    - Think of all the formats: MM-DD-YY, DD-MM-YY, YY-MM-DD, MM/DD/YY, DD/MM/YY, DD/MM/YYYY, 20:45, 8:45am, 8:45 PM, 8:45a, 08:00, 8:10:20, .......
  - Time zones.
  - Daylight savings...
- Thankfully, pandas does a pretty good job here.

In [ ]:
dates_rain = pd.to_datetime(rain_df["Date"])
dates_rain

They are all the same format, so we can also compare dates:

In [ ]:
dates_rain[1] - dates_rain[0] 

In [ ]:
dates_rain[1] > dates_rain[0]

In [ ]:
(dates_rain[1] - dates_rain[0]).total_seconds()

We can also easily extract information from the date columns. 

In [ ]:
dates_rain[1]

In [ ]:
dates_rain[1].month_name()

In [ ]:
dates_rain[1].day_name()

In [ ]:
dates_rain[1].is_year_end

In [ ]:
dates_rain[1].is_leap_year

Above, pandas identified the date column automatically. You can also tell pandas to parse the dates when reading in the CSV:

In [ ]:
rain_df = pd.read_csv(DATA_DIR + "weatherAUS.csv", parse_dates=["Date"])
rain_df.head()

In [ ]:
# Since we re-read the csv file, let's remove the missing targets again
rain_df = rain_df[rain_df["RainTomorrow"].notna()]
rain_df.shape

### <font color='red'>Question 1</font>
- How many time series are present in this dataset? 
- Are the measurements equally spaced? Use the function provided below to help you answer this question.


In [ ]:
def plot_time_spacing_distribution(df, region="Adelaide"):
    """
    Plots the distribution of time spacing for a given region.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame with columns 'Location' and 'Date'.
        region (str): The region (e.g., location) to analyze.
    """
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter data for the given region
    region_data = df[df['Location'] == region]
    
    if region_data.empty:
        print(f"No data available for region: {region}")
        return
    
    # Calculate time differences
    time_diffs = region_data['Date'].sort_values().diff().dropna()
    
    # Count the frequency of each time difference
    value_counts = time_diffs.value_counts().sort_index()
    
    # Display value counts
    print(f"Time spacing counts for {region}:\n{value_counts}\n")
    
    # Plot the bar chart
    plt.bar(value_counts.index.astype(str), value_counts.values, color='skyblue', edgecolor='black')
    plt.title(f"Time Difference Distribution for {region}")
    plt.xlabel("Time Difference (days)")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

### <font color='red'>Question 2</font>

Create train/test splits, using at least 20% samples for the test set. Remember that we should not be calling the usual `train_test_split` with shuffling because if we want to forecast, we aren't allowed to know what happened in the future!

Make sure to call the resulting dataframes `train_df` and `test_df` for the rest of the notebook to work.

### Preprocessing

We have different types of features requiring preprocessing. Let's define a preprocessor with a column transformer. 

This portion of the exercise is given to you, as it is not focused on using temporal information.

- We have missing data. 
- We have categorical features and numeric features. 
- To build a baseline, let's drop the date column and treat this as a usual supervised machine learning problem. 

In [ ]:
numeric_features = [
    "MinTemp",
    "MaxTemp",
    "Rainfall",
    "Evaporation",
    "Sunshine",
    "WindGustSpeed",
    "WindSpeed9am",
    "WindSpeed3pm",
    "Humidity9am",
    "Humidity3pm",
    "Pressure9am",
    "Pressure3pm",
    "Cloud9am",
    "Cloud3pm",
    "Temp9am",
    "Temp3pm",
]
categorical_features = [
    "Location",
    "WindGustDir",
    "WindDir9am",
    "WindDir3pm",
    "RainToday",
]
drop_features = ["Date"]
target = ["RainTomorrow"]

We'll be doing feature engineering and preprocessing features several times. So I've written a function for preprocessing. 

In [ ]:
def preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features,
    target
):

    all_features = set(numeric_features + categorical_features + drop_features + target)
    if set(train_df.columns) != all_features:
        print("Missing columns", set(train_df.columns) - all_features)
        print("Extra columns", all_features - set(train_df.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
    categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
    )

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(train_df)
    ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(
        preprocessor.transform(train_df), index=train_df.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(test_df), index=test_df.index, columns=new_columns
    )

    y_train = train_df["RainTomorrow"]
    y_test = test_df["RainTomorrow"]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [ ]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features, target
)

In [ ]:
# Peek at X_train_enc to see the results of preprocessing
X_train_enc.head()

### <font color='red'>Question 3</font>

Let's treat this as a usual supervised machine learning problem and create a couple of baseline models, to get an idea of what the performance would be if we ignored the time feature.

- Fit and score a `DummyClassifier` (for this exercise, score also on the test set)
- Fit and score a `LogisticRegression` model. 
- Comment on the performance of these baselines.
- Examine the coefficients of the logistic regression model. What features have the biggest impact on the output?

### <font color='red'>Question 4</font>

- Use [`TimeSeriesSplit`](https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split) to carry out cross-validation for a `LogisticRegression` model. 

Some notes before proceeding:
- Things are a bit more complicated here because this dataset has **multiple time series**, one per location. 
- Our approach today will be to ignore the fact that we have multiple time series and just OHE the location
- We'll have multiple measurements for a given timestamp, and that's OK.
- But, `TimeSeriesSplit` expects the dataframe to be sorted by date so let's sort it by date before trying cross-validation.

In [ ]:
train_df_ordered = train_df.sort_values(by=["Date"])
y_train_ordered = train_df_ordered["RainTomorrow"]

In [ ]:
train_df_ordered

In [ ]:
# Cross-validation

### <font color='red'>Question 5</font>

The feature `Date` is probably very useful to predict the target (e.g. different amounts of rain in different seasons) - let's include it as feature!

This is feature engineering!

Think of at least 3 ways to generate features from the `Date` column. Some examples to start:
- Create a column for "days since starting date"
- Using the month as numerical feature, or One-hot encoding it
- One-hot encoding seasons (this requires converting months to seasons - also, remember that seasons are opposite in Australia!)

After adding the new features to the dataset, re-train a `LogisticRegression` model, and see how the performance has changed. Also, observe the coefficients: are the new features important?


### Lag-based features

Realistically, it may be helpful to know if it rained yesterday to predict if it will rain today. Let's add lagged features to our data.

We can "lag" (or "shift") a time series in Pandas with the .shift() method. 

In [ ]:
# Recreating training and test set, to start from a clean slate

train_df = rain_df.query("Date <= 20150630")
test_df = rain_df.query("Date >  20150630")

In [ ]:
# Adding 1 lagged column
train_df = train_df.assign(Rainfall_lag1=train_df["Rainfall"].shift(1))

In [ ]:
train_df[["Date", "Location", "Rainfall", "Rainfall_lag1"]][:20]

**Problem!** We have multiple time series here and we need to be more careful with this. 

When we switch from one location to another we do not want to take the value from the previous location. The function below will help with this task.

In [ ]:
def create_lag_feature(
    df: pd.DataFrame,
    orig_feature: str,
    lag: int,
    groupby: list[str],
    new_feature_name: str | None = None,
    clip: bool = False,
) -> pd.DataFrame:
    """
    Create a lagged (or ahead) version of a feature, optionally per group.

    Assumes df is already sorted by time within each group and has unique indices.

    Parameters
    ----------
    df : pd.DataFrame
        The dataset.
    orig_feature : str
        Name of the column to lag.
    lag : int
        The lag:
          - negative → values from the past (t-1, t-2, ...)
          - positive → values from the future (t+1, t+2, ...)
    groupby : list of str
        Column(s) to group by if df contains multiple time series.
    new_feature_name : str, optional
        Name of the new column. If None, a name is generated automatically.
    clip : bool, default False
        If True, drop rows where the new feature is NaN.

    Returns
    -------
    pd.DataFrame
        A new dataframe with the additional column added.
    """
    if lag == 0:
        raise ValueError("lag cannot be 0 (no shift). Use the original feature instead.")

    # Default name if not provided
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = f"{orig_feature}_lag{abs(lag)}"
        else:
            new_feature_name = f"{orig_feature}_ahead{lag}"

    df = df.copy()

    # Map your convention (negative=past, positive=future) to pandas shift
    # pandas: shift(+k) → past, shift(-k) → future
    periods = abs(lag) if lag < 0 else -lag

    df[new_feature_name] = (
        df.groupby(groupby, sort=False)[orig_feature]
          .shift(periods)
    )

    if clip:
        df = df.dropna(subset=[new_feature_name])

    return df


### <font color='red'>Question 6</font>

- Use `create_lag_feature` to add lagged rainfall features (1 day is enough to start).
- Discuss: would it be ok to add this feature to the test set? If the answer is yes, add it to the test set too.
- Fit and score a `LogisticRegression` model on this new dataset. Compare the results to what was achieved before.
- Observe the coefficients for `Rainfall` and `Rainfall_lag1`. What is their relationship with the target?

Last, here are some more options for lagged features to check out:

- We could also create a lagged version of the target. In fact, this dataset already has that built in! `RainToday` is the lagged version of the target `RainTomorrow`.
- We could also create lagged version of other features, or more lags

In [ ]:
rain_df_modified = create_lag_feature(rain_df, "Rainfall", -1, groupby=["Location"])
rain_df_modified = create_lag_feature(rain_df_modified, "Rainfall", -2, groupby=["Location"])
rain_df_modified = create_lag_feature(rain_df_modified, "Rainfall", -3, groupby=["Location"])
rain_df_modified = create_lag_feature(rain_df_modified, "Humidity3pm", -1, groupby=["Location"])

In [ ]:
rain_df_modified[
    [
        "Date",
        "Location",
        "Rainfall",
        "Rainfall_lag1",
        "Rainfall_lag2",
        "Rainfall_lag3",
        "Humidity3pm",
        "Humidity3pm_lag1",
    ]
].head(10)

Note the pattern of `NaN` values. 

In [ ]:
train_df = rain_df_modified.query("Date <= 20150630")
test_df = rain_df_modified.query("Date >  20150630")

In [ ]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    train_df,
    test_df,
    numeric_features
    + ["Rainfall_lag1", "Rainfall_lag2", "Rainfall_lag3", "Humidity3pm_lag1"],
    categorical_features,
    drop_features,
    target
)

In [ ]:
lr_coef = score_lr_print_coeff(
    preprocessor, train_df, y_train, test_df, y_test, X_train_enc
)

In [ ]:
lr_coef.loc[
    [
        "Rainfall",
        "Rainfall_lag1",
        "Rainfall_lag2",
        "Rainfall_lag3",
        "Humidity3pm",
        "Humidity3pm_lag1",
    ]
]

Note the pattern in the magnitude of the coefficients. 